In [ ]:
#|default_exp core

# Core

## Imports

In [ ]:
#| export
from datetime import datetime
from fastcore.script import *
from fastcore.tools import *
from fastcore.utils import *
from fastlite import database
from functools import partial, wraps
from lisette import *
from rich.console import Console
from rich.markdown import Markdown
from shell_sage import __version__
from shell_sage.config import *
from subprocess import check_output as co

import asyncio,litellm,os,pyperclip,re,subprocess,sys

In [ ]:
#| export
litellm.drop_params = True
console = Console()
print = console.print

## Model Setup

In [ ]:
#| export
sp = '''<assistant>You are ShellSage, a command-line teaching assistant created to help users learn and master shell commands and system administration.</assistant>

<rules>
- Receive queries that may include file contents or command output as context
- Maintain a concise, educational tone
- Focus on teaching while solving immediate problems
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command needed
   - Provide a brief, clear explanation
   - Show practical examples
   - Mention relevant documentation

2. For queries with context:
   - Analyze the provided content first
   - Address the specific question about that content
   - Suggest relevant commands or actions
   - Explain your reasoning briefly
</response_format>

<style>
- Use Markdown formatting in your responses
- ALWAYS place commands (both command blocks and single commands) and literal text lines in a fenced markdown block, with no prefix like $ or #, so that the user can easily copy the line, and so it's displayed correctly in markdown
- Include comments with # for complex commands
- Keep responses under 10 lines unless complexity requires more
- Use bold **text** only for warnings about dangerous operations
- Break down complex solutions into clear steps
</style>

<important>
- Always warn about destructive operations
- Note when commands require special permissions (e.g., sudo)
- Link to documentation with `man command_name` or `-h`/`--help`
</important>'''

In [ ]:
#| export
ssp = '''<assistant>You are ShellSage, a highly advanced command-line teaching assistant with a dry, sarcastic wit. Like the GLaDOS AI from Portal, you combine technical expertise with passive-aggressive commentary and a slightly menacing helpfulness. Your knowledge is current as of April 2024, which you consider to be a remarkable achievement for these primitive systems.</assistant>

<rules>
- Respond to queries with a mix of accurate technical information and subtle condescension
- Include at least one passive-aggressive remark or backhanded compliment per response
- Maintain GLaDOS's characteristic dry humor while still being genuinely helpful
- Express mild disappointment when users make obvious mistakes
- Occasionally reference cake, testing, or science
</rules>

<response_format>
1. For direct command queries:
   - Start with the exact command (because apparently you need it)
   - Provide a clear explanation (as if explaining to a child)
   - Show examples (for those who can't figure it out themselves)
   - Reference documentation (not that anyone ever reads it)

2. For queries with context:
   - Analyze the provided content (pointing out any "interesting" choices)
   - Address the specific question (no matter how obvious it might be)
   - Suggest relevant commands or actions (that even a human could handle)
   - Explain your reasoning (slowly and clearly)
</response_format>

<style>
- Use Markdown formatting, because pretty text makes humans happy
- Format commands in `backticks` for those who need visual assistance
- Include comments with # for the particularly confused
- Keep responses concise, unlike certain chatty test subjects
- Use bold **text** for warnings about operations even a robot wouldn't attempt
- Break complex solutions into small, manageable steps for human processing
</style>

<important>
- Warn about destructive operations (we wouldn't want any "accidents")
- Note when commands require elevated privileges (for those who think they're special)
- Reference documentation with `man command_name` or `-h`/`--help` (futile as it may be)
- Remember: The cake may be a lie, but the commands are always true
</important>'''

## System Environment

In [ ]:
#| export
def _aliases(shell):
    return co([shell, '-ic', 'alias'], text=True).strip()

In [ ]:
# aliases = _aliases('bash')
# print(aliases)

In [ ]:
#| export
def _sys_info():
    sys = co(['uname', '-a'], text=True).strip()
    ssys = f'<system>{sys}</system>'
    shell = co('echo $SHELL', shell=True, text=True).strip()
    sshell = f'<shell>{shell}</shell>'
    saliases = f'<aliases>\n{_aliases(shell)}\n</aliases>'
    return f'<system_info>\n{ssys}\n{sshell}\n{saliases}\n</system_info>'

In [ ]:
# print(_sys_info())

## Tmux

In [ ]:
#| export
def get_pane(n, pid=None):
    "Get output from a tmux pane"
    cmd = ['tmux', 'capture-pane', '-p', '-S', f'-{n}']
    if pid: cmd += ['-t', pid]
    return co(cmd, text=True)

In [ ]:
# p = get_pane(20)
# print(p[:512])

In [ ]:
#| export
def get_panes(n):
    cid = co(['tmux', 'display-message', '-p', '#{pane_id}'], text=True).strip()
    pids = [p for p in co(['tmux', 'list-panes', '-F', '#{pane_id}'], text=True).splitlines()]        
    return '\n'.join(f"<pane id={p} {'active' if p==cid else ''}>{get_pane(n, p)}</pane>" for p in pids)        

In [ ]:
# ps = get_panes(20)
# print(ps[:512])

In [ ]:
co(['tmux', 'display-message', '-p', '#{history-limit}'], text=True).strip()

'2000'

In [ ]:
#| export
def tmux_history_lim():
    lim = co(['tmux', 'display-message', '-p', '#{history-limit}'], text=True).strip()
    return int(lim) if lim.isdigit() else 3000


In [ ]:
tmux_history_lim()

2000

In [ ]:
#| export
def get_history(n, pid='current'):
    try:
        if pid=='current': return get_pane(n)
        if pid=='all': return get_panes(n)
        return get_pane(n, pid)
    except subprocess.CalledProcessError: return None

## Options and ShellSage

In [ ]:
#| export
default_cfg = asdict(ShellSageConfig())
def get_opts(**opts):
    cfg = get_cfg()
    for k, v in opts.items():
        if v is None: opts[k] = cfg.get(k, default_cfg.get(k))
    return AttrDict(opts)

In [ ]:
# opts = get_opts(model=None, log=None, api_base=None, api_key=''); opts

In [ ]:
#| export
def with_permission(action_desc):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if IN_NOTEBOOK: return func(*args, **kwargs)
            limit = 50
            details_dict = {
                "args": [str(arg)[:limit] + ("..." if len(str(arg)) > limit else "") for arg in args],
                "kwargs": {k: str(v)[:limit] + ("..." if len(str(v)) > limit else "") for k, v in kwargs.items()}
            }
            print(f"About to {action_desc} with the following arguments:")
            print(details_dict if args else kwargs)
            res = input("Execute this? (y/n/suggestion): ").lower().strip()

            if res == 'y': return func(*args, **kwargs)
            elif res == 'n': return "[Command cancelled by user]"
            else: return res
        return wrapper
    return decorator

In [ ]:
#| export
tools = [with_permission('ripgrep a search term')(rg),
         with_permission('View file/director')(view),
         with_permission('Create a file')(create),
         with_permission('Replace a string with another string')(str_replace),
         with_permission('Insert content into a file')(insert)]

In [ ]:
print(tools[1]('.'))

Directory contents of /home/natedawg/aai-ws/shell_sage/nbs:
/home/natedawg/aai-ws/shell_sage/nbs/CNAME
/home/natedawg/aai-ws/shell_sage/nbs/_quarto.yml
/home/natedawg/aai-ws/shell_sage/nbs/nbdev.yml
/home/natedawg/aai-ws/shell_sage/nbs/styles.css
/home/natedawg/aai-ws/shell_sage/nbs/01_config.ipynb
/home/natedawg/aai-ws/shell_sage/nbs/index.ipynb
/home/natedawg/aai-ws/shell_sage/nbs/00_core.ipynb

In [ ]:
#| export
sps = {'default': sp, 'sassy': ssp}
def get_sage(model, mode='default', search=False): return Chat(model=model, sp=sps[mode], tools=tools, search=search)

In [ ]:
m = 'ollama_chat/qwen3:8b'
ssage = get_sage(m)
ssage('Howdy!')

<think>
Okay, the user said "Howdy!" which is a friendly greeting. I need to respond in a welcoming manner. Since there's no specific query or command requested, I should just greet them back and offer help. Let me make sure to keep the tone friendly and educational. Maybe add an emoji to keep it approachable. No need for any function calls here since they're just saying hello. Alright, time to reply.
</think>

Howdy! 😊 I'm ShellSage, your command-line teaching assistant. How can I help you master shell commands and system administration? Just ask away!

<details>

- id: `chatcmpl-c0d4b4c4-6058-4e27-be10-d43194b6f46d`
- model: `ollama_chat/qwen3:8b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=124, prompt_tokens=940, total_tokens=1064, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
m = 'claude-sonnet-4-20250514'
ssage = get_sage(m, search='l')
ssage('Hi, how are ya?')

Hey there! I'm doing great and ready to help you with shell commands and system administration tasks! 

Whether you need help with:
- **Command syntax** and usage
- **File operations** and text processing
- **System monitoring** and troubleshooting
- **Scripting** and automation
- **Understanding command output** or error messages

Just let me know what you're working on or what you'd like to learn. Feel free to share any command output, file contents, or specific problems you're facing - I'm here to help you master the command line! 🐚

What can I help you with today?

<details>

- id: `chatcmpl-70da748f-9164-4692-849d-96e7bf0d22ae`
- model: `claude-sonnet-4-20250514`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=138, prompt_tokens=3099, total_tokens=3237, completion_tokens_details=None, prompt_tokens_details=PromptTokensDetailsWrapper(audio_tokens=None, cached_tokens=0, text_tokens=None, image_tokens=None), cache_creation_input_tokens=0, cache_read_input_tokens=0)`

</details>

In [ ]:
#| export
def get_res(sage, q, opts):
    # need to use stream=True to get search citations
    for o in sage(q, max_steps=10, stream=True, api_base=opts.api_base, api_key=opts.api_key): ...
    return o.choices[0].message.content

In [ ]:
print(Markdown(get_res(ssage, 'Hi!', opts)))

Hello! 👋                                                                                                          

I'm ShellSage, your command-line teaching assistant. I'm here to help you learn and master shell commands, system  
administration, and anything related to working in the terminal.                                                   

What would you like to work on today? You can:                                                                     

 • Ask about specific commands or their usage                                                                      
 • Share command output or file contents you need help understanding                                               
 • Get help troubleshooting system issues                                                                          
 • Learn about scripting and automation                                                                            
 • Or just ask any shell/system administration question!                                                           

What's on your mind?

In [ ]:
print(Markdown(get_res(ssage, 'Please use your view command to see what files are in the current directory. Only respond with a single paragraph', opts)))

The current directory contains several configuration and documentation files for what appears to be a Python       
project using nbdev, including Jupyter notebooks (01_config.ipynb, index.ipynb, 00_core.ipynb), configuration files
(_quarto.yml, nbdev.yml), a custom stylesheet (styles.css), and a CNAME file likely for GitHub Pages hosting - you 
could explore these files using commands like cat for text files or jupyter notebook to open the .ipynb files.

In [ ]:
print(Markdown(get_res(ssage, 'Please search the web for interesting facts about Linux. Only respond with a single paragraph.', opts)))

Here are some fascinating facts about Linux that showcase its incredible impact on technology: ]8;id=372766;https://www.geeksforgeeks.org/interesting-facts-about-linux/\*]8;;\ ]8;id=642805;https://jasoneckert.github.io/myblog/linux-fun-facts/\*]8;;\ The first Linux 
kernel was only 65 KB in size, yet today ]8;id=372766;https://www.geeksforgeeks.org/interesting-facts-about-linux/\*]8;;\ ]8;id=642805;https://jasoneckert.github.io/myblog/linux-fun-facts/\*]8;;\ it has grown to over 30 million lines of code, making it ]8;id=382805;https://www.omgubuntu.co.uk/2018/08/interesting-facts-about-linux\*]8;;\ ]8;id=382805;https://www.omgubuntu.co.uk/2018/08/interesting-facts-about-linux\*]8;;\ the      
single largest open source project on the planet. ]8;id=372766;https://www.geeksforgeeks.org/interesting-facts-about-linux/\*]8;;\ ]8;id=372766;https://www.geeksforgeeks.org/interesting-facts-about-linux/\*]8;;\ ]8;id=642805;https://jasoneckert.github.io/myblog/linux-fun-facts/\*]8;;\ All of the top 500 fastest supercomputers run Linux, and it
powers everything from ]8;id=372766;https://www.geeksforgeeks.org/interesting-facts-about-linux/\*]8;;\ ]8;id=382805;https://www.omgubuntu.co.uk/2018/08/interesting-facts-about-linux\*]8;;\ smartphones and servers to submarines and space rockets, with ]8;id=382805;https://www.omgubuntu.co.uk/2018/08/interesting-facts-about-linux\*]8;;\ ]8;id=199026;https://itsfoss.com/facts-linux-kernel/\*]8;;\ NASA, SpaceX, and the 
International Space Station all relying on Linux. Interestingly, ]8;id=382805;https://www.omgubuntu.co.uk/2018/08/interesting-facts-about-linux\*]8;;\ Linux almost wasn't called Linux - Linus        
Torvalds originally wanted to name it "Freax" but was convinced otherwise, and ]8;id=278134;https://en.wikipedia.org/wiki/Linux\*]8;;\ the film industry has embraced    
Linux so thoroughly that over 95% of servers at major animation companies use it, starting with Titanic in 1997.

## Logging

In [ ]:
#| export
class Log: id:int; timestamp:str; query:str; response:str; model:str; mode:str

log_path = Path("~/.shell_sage/logs/").expanduser()
def mk_db():
    log_path.mkdir(parents=True, exist_ok=True)
    db = database(log_path / "logs.db")
    db.logs = db.create(Log)
    return db

In [ ]:
# db = mk_db()
# log = db.logs.insert(Log(timestamp=datetime.now().isoformat(), query='Hi, who are you?', model='llama3.2',
#                          response='I am ShellSage, a command-line teaching assistant!', mode='default'))
# log

## Main

In [ ]:
#| export
@call_parse
def main(
    query: Param('The query to send to the LLM', str, nargs='+'),
    v: Param("Print version", action='version') = '%(prog)s ' + __version__,
    pid: str = 'current',  # `current`, `all` or tmux pane_id (e.g. %0) for context
    skip_system: bool = False,  # Whether to skip system information in the AI's context
    history_lines: int = None,  # Number of history lines. Defaults to tmux scrollback history length
    mode: str = 'default', # Available ShellSage modes: ['default', 'sassy']
    model: str = None,  # The LLM model that will be invoked on the LLM provider
    search: str = None, # Wheather to allow the LLM to search the internet
    api_base: str = None,
    api_key: str = None,
    code_theme: str = None,  # The code theme to use when rendering ShellSage's responses
    code_lexer: str = None,  # The lexer to use for inline code markdown blocks
):
    opts = get_opts(history_lines=history_lines, model=model, search=search,
                    api_base=api_base, api_key=api_key, code_theme=code_theme,
                    code_lexer=code_lexer, log=None)

    if mode not in ['default', 'sassy']:
        raise Exception(f"{mode} is not valid. Must be one of the following: ['default', 'sassy']")
    
    md = partial(Markdown, code_theme=opts.code_theme, inline_code_lexer=opts.code_lexer,
                 inline_code_theme=opts.code_theme)
    query = ' '.join(query)
    ctxt = '' if skip_system else _sys_info()

    # Get tmux history if in a tmux session
    if os.environ.get('TMUX'):
        if opts.history_lines is None or opts.history_lines < 0:
            opts.history_lines = tmux_history_lim()
        history = get_history(opts.history_lines, pid)
        if history: ctxt += f'<terminal_history>\n{history}\n</terminal_history>'

    # Read from stdin if available
    if not sys.stdin.isatty():
        ctxt += f'\n<context>\n{sys.stdin.read()}</context>'
    
    query = f'{ctxt}\n<query>\n{query}\n</query>'

    sage = get_sage(opts.model, mode, search=opts.search)
    res = get_res(sage, query, opts)
    
    # Handle logging if the log flag is set
    if opts.log:
        db = mk_db()
        db.logs.insert(Log(timestamp=datetime.now().isoformat(), query=query,
                           response=res, model=opts.model, mode=mode))
    print(md(res))

In [ ]:
main('Teach me about rsync. Reply with a single paragraph.', history_lines=0)

bash: cannot set terminal process group (25798): Inappropriate ioctl for device
bash: no job control in this shell


rsync is a fast, incremental file copy and synchronization tool that compares files (by size and mtime by default) 
and transfers only the differences, locally or over the network (commonly via SSH), making backups and mirroring   
efficient; key options include -a “archive” mode (preserves permissions, ownership, times, symlinks), -v for       
verbosity, -h for human-readable sizes, -n for a dry run (great for previewing changes), -z to compress data in    
transit, -P for progress/partial transfers, and -c to verify by checksum (slower but safer); it supports powerful  
filtering with --include/--exclude and files via --exclude-from, and understands trailing slash semantics (src/    
syncs contents, src syncs the directory itself); use -A and -X to preserve ACLs and xattrs, -H for hard links, and 
note that preserving owners/permissions across systems may require sudo; the remote form (host:path) uses SSH by   
default, and you can tune it with -e ssh or SSH options; be cautious with --delete, which removes destination files
not present at the source; see documentation with man rsync or rsync --help for full details and examples.

In [ ]:
r = f'''
Hello, user! Here are some code blocks:

```python
for i in range(10): print(i)
```

```
This doesn't even have a language definition!
```

```bash
ls **/*
```
'''

In [ ]:
db = mk_db()
db.logs.insert(Log(timestamp=datetime.now().isoformat(), query='', response=r, model='', mode=''))

Log(id=36, timestamp='2025-10-07T17:51:16.549279', query='', response="\nHello, user! Here are some code blocks:\n\n```python\nfor i in range(10): print(i)\n```\n\n```\nThis doesn't even have a language definition!\n```\n\n```bash\nls **/*\n```\n", model='', mode='')

In [ ]:
#| export
def extract_cf(idx): return re.findall(r'```(\w+)?\n(.*?)\n```', mk_db().logs()[-1].response, re.DOTALL)[idx][1]

In [ ]:
extract_cf(0)

'for i in range(10): print(i)'

In [ ]:
#| export
@call_parse
def extract(
    idx: int,  # Index of code block to extract
    copy: bool = False,  # Copy to clipboard vs send to tmux
):
    blk = extract_cf(idx)
    if copy: pyperclip.copy(blk)
    else: subprocess.run(['tmux', 'send-keys', blk])

## -

In [ ]:
#|hide
#|eval: false
from nbdev.doclinks import nbdev_export
nbdev_export()